# Fitten eines Crash Modells

Die Idee ist es, ein Modell zu erzeugen, das bei gegebenem Ort und Zeit eine Vorhersage über die Unfallhäufigkeit treffen kann. Dieses Modell könnte dann ausgeführt werden, wenn sich jemand ein Fahrrad ausleiht und durch die Kombination der Modellvorhersage und den Stationstraffic kann ein Preis für die ihm vorgeschlagene Versicherung ermittelt werden.

In [2]:
from datasets.bike_crash_dataset import BikeCrashDataset
from datasets.citibike_dataset import CitibikeDataset

crash_dataset = BikeCrashDataset("data/bike_crashes.csv")
citibike_dataset = CitibikeDataset("data/2023-citibike-tripdata/202312-citibike-tripdata/202312-citibike-tripdata_1.csv")

crash_dataset.citibike_alignment(citibike_dataset)

c:\Users\JanSchmitz\code\axa_challenge\datasets\citibike_dataset.py:37: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


In [21]:
df_raster = crash_dataset.get_spatio_temporal_rasterization(bins=80, time_bin_size=60)

In [22]:
from sklearn.model_selection import train_test_split

X = df_raster[['x_center', 'y_center', 'time_center']].values
y = df_raster['crash_count'].values

# Split into training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Die erste Idee war einen Gaußprozess zu nutzen, da dieser auch die Unsicherheit mit abbildet, die vor allem für die Anwendung im Versicherungskontext relevant ist. Für diesen ist die Datenmenge jedoch zu groß. 

In [67]:
rf_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42) 
rf_model.fit(X_train, y_train)


RandomForestRegressor(max_depth=10, random_state=42)

In [26]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def evaluate_model(y_true, y_pred, model_name="Model"):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"{model_name} Evaluation:")
    print(f"  MAE: {mae:.3f}")
    print(f"  MSE: {mse:.3f}")
    print(f"  R^2: {r2:.3f}\n")

In [68]:
import numpy as np

y_pred_rf = rf_model.predict(X_test)

evaluate_model(y_test, y_pred_rf, model_name="Random Forest")
evaluate_model(y_test, np.zeros(y_test.shape), model_name="Zeros predicted")
evaluate_model(y_test, np.ones(y_test.shape), model_name="Ones predicted")

Random Forest Evaluation:
  MAE: 0.884
  MSE: 1.660
  R^2: 0.302

Zeros predicted Evaluation:
  MAE: 1.990
  MSE: 6.340
  R^2: -1.665

Ones predicted Evaluation:
  MAE: 0.990
  MSE: 3.360
  R^2: -0.412



Das Random Forest Modell ist zumindest schon mal besser als die Vorhersage von konstanten Crash Zahlen. 

Vergleich mit anderen Modellen und exemplarische Suche nach guten Hyperparametern:

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR

models = [
    {
        'name': 'Random Forest',
        'estimator': RandomForestRegressor(random_state=42),
        'param_grid': {
            'n_estimators': [50, 100, 150],
            'max_depth': [None, 5, 10, 15]
        }
    },
    {
        'name': 'Gradient Boosting',
        'estimator': GradientBoostingRegressor(random_state=42),
        'param_grid': {
            'n_estimators': [100, 200, 300],
            'learning_rate': [0.1, 0.05, 0.01],
            'max_depth': [3, 5, 7]
        }
    },
    {
        'name': 'SVR',
        'estimator': SVR(),
        'param_grid': {
            'C': [0.1, 1, 10, 100],
            'epsilon': [0.01, 0.1, 1],
            'gamma': ['scale', 'auto']
        }
    }
]

for model in models:
    print("Fitting", model['name'])
    grid = GridSearchCV(model['estimator'], model['param_grid'], cv=3,
                        scoring='neg_mean_squared_error', n_jobs=-1)
    grid.fit(X_train, y_train)
    print("Best parameters:", grid.best_params_)
    
    best_model = grid.best_estimator_
    model['best_model'] = best_model
    y_pred = best_model.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print(f"{model['name']} Evaluation:")
    print(f"  MAE: {mae:.3f}")
    print(f"  MSE: {mse:.3f}")
    print(f"  R²: {r2:.3f}\n")


Fitting Random Forest
Best parameters: {'max_depth': 10, 'n_estimators': 150}
Random Forest Evaluation:
  MAE: 0.884
  MSE: 1.660
  R²: 0.302

Fitting Gradient Boosting
Best parameters: {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200}
Gradient Boosting Evaluation:
  MAE: 0.849
  MSE: 1.528
  R²: 0.358

Fitting SVR
Best parameters: {'C': 100, 'epsilon': 1, 'gamma': 'scale'}
SVR Evaluation:
  MAE: 1.041
  MSE: 2.142
  R²: 0.100



In diesem Fall scheint Gradient Boosting am besten zu sein. 

Es könnten noch mehr Modelle getestet und Verbesserungen durchgeführt werden. Für die Challenge wird angenommen, dass ein Modell ausreichend gut ist. Dieses wird Exportiert und kann zur Bestimmung der Versicherungssumme verwendet werden.

In [59]:
import pickle

filename = "./fitted_models/crash_model.pkl"
pickle.dump(models[1]['best_model'], open(filename, 'wb'))